setup and import files into DataInstanceLibrary

In [1]:
from __future__ import annotations
from dataclasses import dataclass, field

from metasmith.models.solver import Endpoint, Namespace
from metasmith.models.libraries import *

from local.utils import LinkifyPath
from local.constants import WORKSPACE_ROOT
CACHE = WORKSPACE_ROOT/"main/local_mock/cache"

In [2]:
_types = DataTypeLibrary(
    source = WORKSPACE_ROOT/"main/local_mock/prototypes/metagenomics.dev3.yml",
)
_types["contigs"] = DataType(dict(
    format="FASTA",
    data="DNA sequence"
))
_types["orfs_faa"] = DataType(dict(
    format="FASTA",
    data="Amino acid sequence"
))
_types["protein_reference_diamond"] = DataType(dict(
    format=".dmnd",
    data="database reference"
))
_types["oci_image_diamond"] = DataType(dict(
    format="OCI",
    data="software contaienr",
    provides=["diamond"]
))
_types["oci_image_pprodigal"] = DataType(dict(
    format="OCI", 
    data="software contaienr",
    provides=["pprodigal"]
))
_types["orf_annotations"] = DataType(dict(
    format="CSV",
    data="Protein features"
))
_types.Save()
LinkifyPath(_types.source)
types = DataTypeLibrary.Load(_types.source)
len(types)

./../../main/local_mock/prototypes/metagenomics.dev3.yml


6

In [3]:
xgdb_path = CACHE/"test.xgdb"
_xgdb = DataInstanceLibrary(xgdb_path)
refdb = DataInstanceLibrary(CACHE/"ref.xgdb")
_xgdb.ImportDataInstance(WORKSPACE_ROOT/"scratch/test_ws/data/local/example.fna", types["contigs"], overwrite=True)
refdb.ImportDataInstance(WORKSPACE_ROOT/"scratch/test_ws/data/local/uniprot_sprot.dmnd", types["protein_reference_diamond"], overwrite=True)
refdb.ImportDataInstance(WORKSPACE_ROOT/"scratch/test_ws/data/local/diamond.oci.uri", types["oci_image_diamond"], overwrite=True)
refdb.ImportDataInstance(WORKSPACE_ROOT/"scratch/test_ws/data/local/pprodigal.oci.uri", types["oci_image_pprodigal"], overwrite=True)
_xgdb.Save()
refdb.Save()
LinkifyPath(_xgdb._index_path())
xgdb = DataInstanceLibrary.Load(_xgdb.source)

./../../main/local_mock/cache/test.xgdb/info.yml


In [4]:
trlib = TransformInstanceLibrary.Load([
    Path("./transforms/simple_1"),
    # Path("./transforms/dupe_test"),
])
for k, v in trlib:
    print(k.name, type(v))

diamond.py <class 'metasmith.models.libraries.TransformInstance'>
pprodigal.py <class 'metasmith.models.libraries.TransformInstance'>


In [5]:
xgdb.manifest

{PosixPath('example.fna'): DataInstance(source=PosixPath('example.fna'), type=DataType(properties={'data': 'DNA sequence', 'format': 'FASTA'}, lineage=[]))}

In [6]:
from metasmith.models.solver import Transform
from metasmith.solver import _solve_by_bounded_dfs

@dataclass
class WorkflowStep:
    key: str
    transform_key: str
    uses: list[tuple[DataInstance, Endpoint]]
    produces: list[tuple[DataInstance, Endpoint]]
    transform: TransformInstance | Path

@dataclass
class WorkflowPlan:
    given: list[tuple[DataInstance, Endpoint]]
    targets: list[tuple[DataInstance, Endpoint]]
    steps: list[WorkflowStep]

    def __len__(self):
        return len(self.steps)
    
    @classmethod
    def Generate(
        cls,
        given: Iterable[DataInstanceLibrary], transforms: TransformInstanceLibrary, targets: DataTypeLibrary,
        seed: int=42,
    ):
        _namespace = Namespace(seed=seed)
        _given = given.AsEndpoints(_namespace)
        # _target = targets.AddAsDependencies(Transform(_namespace))
        # _transforms = transforms.AsTransforms(_namespace)

        # solutions = _solve_by_bounded_dfs(
        #     _given.keys(),


        # )

        pass

given=[xgdb, refdb]
transforms=trlib
targets=[
    types["orf_annotations"].WithLineage([
        # xgdb.Get(types["contigs"]),
        xgdb["example.fna"].type,
    ]),
]
seed=42
_run_path = CACHE/"ws1/run_dev3"
_run_path.mkdir(parents=True, exist_ok=True)
_namespace = Namespace(seed=seed)
_given_dict: dict[Endpoint, DataInstance] = {}
for _map in [lib.AsEndpoints(_namespace) for lib in given]:
    _given_dict.update(_map)




_given_dict

AttributeError: 'DataType' object has no attribute 'AsEndpoint'